### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [4]:
## Create a Keras classifier
model = KerasClassifier(
    build_fn=create_model,
    neurons=32,
    layers=1,
    epochs=50,
    batch_size=32,
    verbose=1
)


In [5]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64],
    'layers': [1, 2, 3],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/Users/jaybhesania/completemachinelearning/ANN Salary Regression/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jaybhesania/completemachinelearning/ANN Salary Regression/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jaybhesania/completemachinelearning/ANN Salary Regression/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jaybhesania/completemachinelearning/ANN Salary Regression/venv/lib/python3.11/site-packages/scikeras/w

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2025-05-09 18:45:51.295791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.372960: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.376014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.376815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.378656: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.379459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-05-09 18:45:51.390870: I tensorflow/core/grappler/optimizers/cust

200/200 [==============================] - 7s 10ms/step - loss: 0.5648 - accuracy: 0.7131
Epoch 2/50
200/200 [==============================] - 7s 10ms/step - loss: 0.5404 - accuracy: 0.7486
Epoch 2/50
Epoch 2/50
200/200 [==============================] - 7s 10ms/step - loss: 0.5983 - accuracy: 0.7017
Epoch 2/50
200/200 [==============================] - 7s 11ms/step - loss: 0.5868 - accuracy: 0.6992
Epoch 2/50
200/200 [==============================] - 2s 11ms/step - loss: 0.4353 - accuracy: 0.8102
Epoch 3/50
200/200 [==============================] - 2s 11ms/step - loss: 0.4437 - accuracy: 0.8109
Epoch 3/50
200/200 [==============================] - 2s 11ms/step - loss: 0.4485 - accuracy: 0.8072
Epoch 3/50
200/200 [==============================] - 2s 12ms/step - loss: 0.4368 - accuracy: 0.8097
Epoch 3/50
200/200 [==============================] - 2s 10ms/step - loss: 0.4321 - accuracy: 0.8112
Epoch 4/50
200/200 [==============================] - 2s 10ms/step - loss: 0.4347 - accurac

...

200/200 [==============================] - 3s 16ms/step - loss: 141.6384 - accuracy: 0.7236
Epoch 41/100
200/200 [==============================] - 3s 16ms/step - loss: 274.6768 - accuracy: 0.6353
Epoch 94/100
200/200 [==============================] - 3s 16ms/step - loss: 86.7905 - accuracy: 0.7036
Epoch 69/100
200/200 [==============================] - 3s 16ms/step - loss: 50.9396 - accuracy: 0.7038
Epoch 100/100
200/200 [==============================] - 3s 17ms/step - loss: 568.1815 - accuracy: 0.6925
Epoch 47/100
200/200 [==============================] - 3s 16ms/step - loss: 238.0369 - accuracy: 0.6502
Epoch 95/100
200/200 [==============================] - 3s 16ms/step - loss: 131.4547 - accuracy: 0.6764
Epoch 70/100
200/200 [==============================] - 4s 15ms/step - loss: 0.4797 - accuracy: 0.7814
Epoch 2/100
200/200 [==============================] - 4s 22ms/step - loss: 55.5200 - accuracy: 0.7059
Epoch 49/100
200/200 [==============================] - 5s 25ms/step - lo

/Users/jaybhesania/completemachinelearning/ANN Salary Regression/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2025-05-09 20:48:57.381457: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-05-09 20:48:57.385054: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-05-09 20:48:57.385662: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-05-09 20:48:57.385913: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-09 20:48:57.385936: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0

Epoch 1/50


2025-05-09 20:48:57.927131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 2s 4ms/step - loss: 0.6020 - accuracy: 0.7059
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4446 - accuracy: 0.8081
Epoch 3/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4353 - accuracy: 0.8108
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4334 - accuracy: 0.8098
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4336 - accuracy: 0.8099
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4333 - accuracy: 0.8131
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4333 - accuracy: 0.8095
Epoch 8/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4329 - accuracy: 0.8102
Epoch 9/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4334 - accuracy: 0.8096
Epoch 10/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4331 - accuracy: 0.8133
Epoch 11/5